In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from sklearn.metrics.pairwise import cosine_similarity
import onnx
import cv2
import numpy as np
import onnxruntime
import pandas as pd
from tqdm import tqdm
import dask.dataframe as dd

In [5]:
class EmbeddingDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        # Преобразуем строковое представление списков обратно в NumPy массивы
        # self.data['emb_r50'] = self.data['emb_r50'].apply(lambda x: np.array(eval(x)))
        # self.data['emb_r100'] = self.data['emb_r100'].apply(lambda x: np.array(eval(x)))
        self.labels = self.data['label'].values

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        emb_r50 = np.array(eval(self.data.iloc[idx]['emb_r50']))
        emb_r100 = np.array(eval(self.data.iloc[idx]['emb_r100']))
        label = self.labels[idx]
        # Конвертируем эмбеддинги в тензоры
        emb_r50 = torch.tensor(emb_r50, dtype=torch.float32).squeeze(0)
        emb_r100 = torch.tensor(emb_r100, dtype=torch.float32).squeeze(0)
        return emb_r50, emb_r100, label

In [6]:
batch_size = 256

# Создаём датасеты из CSV файлов
train_dataset = EmbeddingDataset('train_embeddings.csv')
test_dataset = EmbeddingDataset('test_embeddings.csv')

In [7]:

# Создаём DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [8]:
class EmbeddingMapper(nn.Module):
    def __init__(self, input_dim=512, output_dim=512):
        super(EmbeddingMapper, self).__init__()
        # self.model = nn.Sequential(
        #     nn.Linear(input_dim, 1024),
        #     nn.ReLU(),
        #     nn.BatchNorm1d(1024),
        #     nn.Linear(1024, 1024),
        #     nn.ReLU(),
        #     nn.BatchNorm1d(1024),
        #     nn.Linear(1024, output_dim)
        # )
        self.lin1 = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024))
        self.lin2 = nn.Sequential(
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024))
        self.lin3 = nn.Linear(1024, output_dim)
        
    def forward(self, x):
        # print(x.shape)
        x = self.lin1(x)
        x = self.lin2(x)
        x = self.lin3(x)
        return x

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = EmbeddingMapper(input_dim=512, output_dim=512).to(device)
criterion = nn.MSELoss()

# Устанавливаем weight_decay для L2-регуляризации
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

In [10]:
def cosine_similarity_torch(x1, x2):
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    return cos(x1, x2)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    print(f'Epoch {epoch+1}/{num_epochs}')
    for inputs, targets, _ in tqdm(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)

    # Оценка на тестовой выборке
    model.eval()
    with torch.no_grad():
        test_loss = 0.0
        total_cosine_sim = 0.0
        total_samples = 0

        for inputs, targets, _ in test_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item() * inputs.size(0)

            cosine_sim = cosine_similarity_torch(outputs, targets)
            total_cosine_sim += cosine_sim.sum().item()
            total_samples += inputs.size(0)

        test_loss = test_loss / len(test_loader.dataset)
        avg_cosine_sim = total_cosine_sim / total_samples
    
    torch.save(model.state_dict(), f'checkpoints/embedding_mapper{epoch}.pth')

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, '
          f'Test Loss: {test_loss:.4f}, Cosine Similarity: {avg_cosine_sim:.4f}')

Epoch 1/10


100%|██████████| 1753/1753 [15:43<00:00,  1.86it/s]


Epoch 1/10, Train Loss: 0.4355, Test Loss: 0.3021, Cosine Similarity: 0.8622
Epoch 2/10


100%|██████████| 1753/1753 [16:09<00:00,  1.81it/s]


Epoch 2/10, Train Loss: 0.2764, Test Loss: 0.2761, Cosine Similarity: 0.8740
Epoch 3/10


...

Epoch 6/10, Train Loss: 0.2512, Test Loss: 0.2575, Cosine Similarity: 0.8820
Epoch 7/10


100%|██████████| 1753/1753 [23:00<00:00,  1.27it/s]


Epoch 7/10, Train Loss: 0.2482, Test Loss: 0.2537, Cosine Similarity: 0.8837
Epoch 8/10


 96%|█████████▌| 1681/1753 [18:17<00:47,  1.53it/s]


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), 'embedding_mapper.pth')

In [ ]:
def map_embedding(emb_r50):
    emb_r50 = torch.from_numpy(emb_r50).float().to(device)
    with torch.no_grad():
        emb_mapped = model(emb_r50.unsqueeze(0))
    return emb_mapped.cpu().numpy().flatten()

In [12]:
from huggingface_hub import hf_hub_download

hf_hub_download(repo_id="FoivosPar/Arc2Face", filename="arc2face/config.json", local_dir="./models")
hf_hub_download(repo_id="FoivosPar/Arc2Face", filename="arc2face/diffusion_pytorch_model.safetensors", local_dir="./models")
hf_hub_download(repo_id="FoivosPar/Arc2Face", filename="encoder/config.json", local_dir="./models")
hf_hub_download(repo_id="FoivosPar/Arc2Face", filename="encoder/pytorch_model.bin", local_dir="./models")

diffusion_pytorch_model.safetensors:   2%|1         | 62.9M/3.44G [00:00<?, ?B/s]

encoder/config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/492M [00:00<?, ?B/s]

'models\\encoder\\pytorch_model.bin'